In [2]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import importlib

# keeping company information in additional file
import data_file
importlib.reload(data_file)

# Get current year and month
now = datetime.now()

# Subtract one month from current date
last_month = now - relativedelta(months=1)  #

# Format last month as string in YYYYMM format
year_month = last_month.strftime("%Y%m")

print(year_month)

# Construct file path
file_path = f"data_files/{year_month}.xlsx"

# Read xlsx file into pandas DataFrame
df = pd.read_excel(file_path)

202306


In [3]:
#combining products groups 
control = data_file.control
transmitters = data_file.transmitters
flowmeters = data_file.flowmeters
gc = data_file.gc
analytical = data_file.analytical
asi = data_file.asi
other_prj_product = data_file.other_prj_product

all_groups = control + transmitters + flowmeters + gc + analytical + asi + other_prj_product

In [4]:
year_month_int = int(year_month)
df = df[df['year_month'] == year_month_int]

In [5]:
my_company_name = data_file.my_company_name
exclusion_company_one = data_file.exclusion_company_one
exclusion_company_two = data_file.exclusion_company_two

last_column_name = 'order_intake_amount_eur'
df = df.rename(columns={df.columns[-1]: last_column_name})

#necessary modifications of columns
df['bu'] = df['bu'].astype(str)
df['company_up'] = df['sold_to_customer_n'].str.upper()


# Filter out rows with unwanted value in column2
df = df[~df['company_up'].str.contains(my_company_name) & ~df['company_up'].str.contains(exclusion_company_one) & ~df['company_up'].str.contains(exclusion_company_two)]  

In [6]:
# Drop rows where 'ColumnA' value is 0
df = df.drop(df[df[last_column_name] == 0].index)

In [7]:
# Create an ExcelWriter object
writer = pd.ExcelWriter(f'data_files/outcome/df_wo_{my_company_name}_{year_month}.xlsx')
# Save each DataFrame to a separate sheet in the same file
df.to_excel(writer, sheet_name='report_bu')
# Save the file
writer.close()

In [8]:
# Define a function to combine values in bu
def combine_values(x):
    if x in control:
        return f'Control {control}'
    elif x in transmitters:
        return f'Transmitters {transmitters}'
    elif x in flowmeters:
        return f'Flowmeters {flowmeters}'
    elif x in gc:
        return f'GC {gc}'
    elif x in analytical:
        return f'Analytical Products {analytical}'
    elif x in asi:
        return f'ASI {asi}'
    elif x in other_prj_product:
        return f'Other prj products {other_prj_product}'
    else:
        return 'Other'

# Apply the function to create a new column with combined values
df['Business Unit'] = df['bu'].apply(combine_values)

# Group the data by the new combined column and sum the values in order_intake_amount_eur
report_bu = df.groupby(['Business Unit'])['order_intake_amount_eur'].agg('sum')
report_bu = report_bu.div(1000).round(1)

# Compute the total excluding the value Other in Business Unit
report_bu = report_bu.loc[report_bu.index.get_level_values('Business Unit') != 'Other'].groupby('Business Unit').agg('sum').to_frame('Total, kEuro')

# Append the total row to the bottom of the grouped DataFrame
report_bu = report_bu.reset_index()
col_sum = report_bu['Total, kEuro'].sum()
report_bu.loc[len(report_bu)] = ['Total: ', col_sum]

report_opco = df.groupby(['company_code_n', 'bu'])['order_intake_amount_eur'].sum()

# Drop rows with 0 values for 'order_intake_amount_eur'
report_opco = report_opco[report_opco != 0]


# Divide the sums by 1000 and round them
report_opco = report_opco.div(1000).round(1)
# Sort the values in descending order
report_opco = report_opco.sort_values(ascending=False)
report_opco = report_opco.reset_index()
# keeping only product bu groups
report_opco = report_opco[report_opco['bu'].isin(all_groups)]

In [9]:
# Create an ExcelWriter object
writer = pd.ExcelWriter(f'data_files/outcome/bu_summary_{year_month}.xlsx')

# Save each DataFrame to a separate sheet in the same file
report_bu.to_excel(writer, sheet_name='report_bu')
report_opco.to_excel(writer, sheet_name='report_opco')

# Save the file
writer.close()